In [15]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

path = "kaggle_3m"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(device)

import os
import pandas as pd
from PIL import Image
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset

class ImageFolderDataset(Dataset):
    def __init__(self, root_dir, data_frame):
        self.root_dir = root_dir
        self.labels = data_frame
        self.labels.dropna(subset=['tumor_location'], inplace=True)
        self.transform = ToTensor()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        label = self.labels.loc[idx, 'tumor_location']
        folder_name = self.labels.iloc[idx, 0]

        images = []

        for dir_name in os.listdir(self.root_dir):
            if dir_name.startswith(folder_name):
                img_folder = os.path.join(self.root_dir, dir_name)

                for img_name in sorted(os.listdir(img_folder)):
                    if not img_name.endswith('mask.tif'):
                        img_path = os.path.join(img_folder, img_name)

                        image = Image.open(img_path)
                        image = self.transform(image)
                        images.append(image)

        images = torch.stack(images)

        return images.float().to(device), torch.tensor([label]).long().to(device)



df = pd.read_csv('kaggle_3m/data.csv')
df.dropna(subset=['tumor_location'], inplace=True)
class_mapping = {1: 0, 2: 1, 4: 2, 5: 3, 6: 4}
df = df.replace({'tumor_location' : class_mapping})
print(df['tumor_location'].unique())

dataset = ImageFolderDataset(root_dir='kaggle_3m', data_frame = df)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


cpu
[1. 4. 3. 2. 0.]


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 6, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(64*64*6, 256)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(256, 128, batch_first=True)
        self.fc1 = nn.Linear(128, 5)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc1(x[:, -1, :])
        return x

class CombinedModel(nn.Module):
    def __init__(self):
        super(CombinedModel, self).__init__()
        self.cnn = CNN()
        self.lstm = LSTM()

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        x = x.view(timesteps, C, H, W)
        x = self.cnn(x)
        x = x.view(batch_size, timesteps, -1)
        x = self.lstm(x)
        return x

model = CombinedModel().to(device)

print(model)



CombinedModel(
  (cnn): CNN(
    (conv1): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(5, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fc): Linear(in_features=24576, out_features=256, bias=True)
  )
  (lstm): LSTM(
    (lstm): LSTM(256, 128, batch_first=True)
    (fc1): Linear(in_features=128, out_features=5, bias=True)
  )
)


In [17]:
import torch.optim as optim

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 3
for epoch in range(num_epochs):
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(train_loader):

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.squeeze(0))

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss.backward()
        optimizer.step()

        torch.cuda.empty_cache()

    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Accuracy: {accuracy}%')

print("Training complete.")



Epoch 1/3, Loss: 0.3229725956916809, Accuracy: 52.87356321839081%
Epoch 2/3, Loss: 1.2681533098220825, Accuracy: 58.62068965517241%
Epoch 3/3, Loss: 0.5151243209838867, Accuracy: 58.62068965517241%
Training complete.


In [18]:
correct = 0
total = 0

model.eval()

for inputs, labels in val_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the validation data: {accuracy}%')

Accuracy of the model on the validation data: 54.54545454545455%
